In [12]:
# Imports
from azure.common.credentials import ServicePrincipalCredentials
from azure.identity import ClientSecretCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.storage import StorageManagementClient
from azureml.core import Workspace
# from azure.ai.ml.entities import Workspace

import yaml

In [3]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

subscription_id = config.get("subscription_id")
client_id = config.get("client_id")
secret = config.get("secret")
tenant_id = config.get("tenant_id")


In [22]:
credential = ClientSecretCredential(
    client_id=client_id,
    client_secret=secret,
    tenant_id=tenant_id
)

from azureml.core.authentication import ServicePrincipalAuthentication

credential_aml = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=client_id,
    service_principal_password=secret)

# credential_sp = ServicePrincipalCredentials(
#     client_id=client_id,
#     secret=secret,
#     tenant=tenant_id
# )

In [5]:
# Configuration Parameters
resource_group_name = 'AI_IC_NAM_GenAI-Template-1'
location = 'eastus'
vnet_name = 'MyVNet'
subnet_name = 'MySubnet'
aml_workspace_name = 'genai-test-workspace'
storage_account_name = 'genaitemplate1storage'
openai_account_name = 'genai-test-openai'

resource_client = ResourceManagementClient(credential, subscription_id)
network_client = NetworkManagementClient(credential, subscription_id)
storage_client = StorageManagementClient(credential, subscription_id)

In [6]:
# Create Resource Group
resource_client.resource_groups.create_or_update(resource_group_name, {'location': location})

In [7]:
# Create Virtual Network and Subnet
vnet_params = {
    'location': location,
    'address_space': {
        'address_prefixes': ['10.0.0.0/16']
    }
}
network_client.virtual_networks.begin_create_or_update(resource_group_name, vnet_name, vnet_params)

In [8]:
subnet_params = {
    'address_prefix': '10.0.0.0/24',
    'service_endpoints': [{'service': 'Microsoft.CognitiveServices'}]
}
network_client.subnets.begin_create_or_update(resource_group_name, vnet_name, subnet_name, subnet_params)

In [9]:
# Add Network Rule for Subnet
subnet = network_client.subnets.get(resource_group_name, vnet_name, subnet_name)
subnet_id = subnet.id

In [10]:
# Create Blob Storage
storage_params = {
    'sku': {'name': 'Standard_LRS'},
    'kind': 'StorageV2',
    'location': location,
    'network_acls': {'default_action': 'Deny'}
}
storage_client.storage_accounts.begin_create(resource_group_name, storage_account_name, storage_params)

In [24]:

# Create AML Workspace with VNet
aml_workspace = Workspace.create(
    name=aml_workspace_name,
    auth=credential_aml,
    subscription_id=subscription_id,
    resource_group=resource_group_name,
    location=location,
    create_resource_group=False,
    exist_ok=True,
    friendly_name=aml_workspace_name,
    # disable_public_network_access=True  # Disable public network
    # private_endpoint_config={
    #     'name': 'MyPrivateEndpoint',
    #     'subnet': {
    #         'id': subnet_id
    #     }
    # }
)


TypeError: Workspace.create() got an unexpected keyword argument 'disable_public_network_access'

In [23]:
workspace = Workspace(workspace_name=aml_workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group_name,
                      auth=credential_aml)





In [ ]:
allow_public_access_when_behind_vnet=None


In [18]:
from azureml.core import ComputeTarget
# aml_compute = ComputeTarget(workspace=workspace, name='<AML Compute Name>')


In [ ]:
# Create AML Compute Cluster within VNet
try:
    compute_target = ComputeTarget(workspace=aml_workspace, name='MyAmlCompute')
    print('Found existing cluster.')
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',
                                                   min_nodes=0,
                                                   max_nodes=2,
                                                   vnet_resourcegroup_name=resource_group_name,

In [ ]:

# Create AML Workspace
aml_workspace = Workspace.create(
    name=aml_workspace_name,
    subscription_id='<subscriptionID>',
    resource_group=resource_group_name,
    location=location,
    create_resource_group=False,
    exist_ok=True
)







# Create OpenAI Account (Emulate since OpenAI specific SDK not available)
# Replace with your specific logic using Azure OpenAI commands or SDK
openai_params = {
    'sku': {'name': 'S0'},
    'kind': 'OpenAI',
    'location': location,
    'network_acls': {'default_action': 'Deny'}
}
# (Replace with real command to create OpenAI account)

# Add Network Rule for Subnet
subnet = network_client.subnets.get(resource_group_name, vnet_name, subnet_name)
subnet_id = subnet.id
# (Replace with real command to add network rule for OpenAI account using subnet_id)

print("Setup complete.")
